# Poland 2D Line 001 number of samples
Compare the number of samples indicated in the file header against the number of samples in each trace to determine if they are consistent. 
This example uses the open source 2D land vibroseis line from [SEG Wiki](https://wiki.seg.org/wiki/2D_Vibroseis_Line_001).

In [1]:
import os
from pathlib import Path
import logging
from datetime import datetime
from poland2d_context import segytools

TEXTENCODE = 'ebcdic'
BYTEORDER = '>'

# -- logging
logging.basicConfig(level=logging.DEBUG)
now = datetime.now() # current date and time

logger = logging.getLogger("line001_segy_fileheader")
# handler = logging.StreamHandler()
# formatter = logging.Formatter('%(asctime)s : %(name)s : %(levelname)s : %(message)s')
# handler.setFormatter(formatter)
# logger.addHandler(handler)

segyfile = "data/Line_001.sgy"
pathsegyfile = Path(str(segyfile))
assert (pathsegyfile.is_file())

segyfilesize = os.path.getsize(segyfile)

file_header = segytools.SegyFileHeaderRev2(segy_logger=logger)
trace_header = segytools.SegyTraceHeaderRev2()

# 'rb' is "read bytes"
with open(segyfile, 'rb') as fobj:
    # read the first 3200 bytes.
    # This will always be 3200 byte textual file header
    b_text_header = fobj.read(3200)

    b_file_header = fobj.read(file_header.byte_length)
    file_header.set_header_values(buf=b_file_header, byteorder=BYTEORDER)
    num_samp = file_header.num_samples_per_trace.value
    #--print(f'number of samples in file header: {num_samp}')
    logger.info(f"number of samples in file header: {num_samp}")
    smp_int_micros = file_header.sample_interval.value
    #--print(f'sample interval (microseconds) in file header: {smp_int_micros}')
    logger.info(f"sample interval (microseconds) in file header: {smp_int_micros}")
    fixed_len_flag = file_header.fixed_length.value
    if fixed_len_flag == 1:
        #--print('All traces should have the same trace length.')
        logger.info("All traces should have the same trace length.")
    else:
        #--print('Trace length may not be fixed.')
        logger.info("Trace length may not be fixed.")

    sample_size_in_bytes = file_header.sample_format_size_in_bytes()
    trc_data_length_in_bytes = file_header.num_samples_per_trace.value * sample_size_in_bytes
    #--print(f'trace data length in bytes: {trc_data_length_in_bytes}')
    logger.info(f"number of bytes in each trace: {trc_data_length_in_bytes}")

    # Loop through traces reading the number of samples and sample interval for each trace.
    # Note that if the number of samples changes, the trace data length in bytes will also 
    # change. This can happen when auxiliary channels are present which are shorter or 
    # longer than the "normal" trace data.
    while fobj.tell() < segyfilesize:
        b_trace_header = fobj.read(trace_header.byte_length)
        trace_header.set_header_values(buf=b_trace_header, byteorder=BYTEORDER)
        if trace_header.num_samples.value != file_header.num_samples_per_trace.value:
            logger.info(f"trace header samples {trace_header.num_samples.value} does not match file header samples {file_header.num_samples_per_trace.value}")
            # trc_data_length_in_bytes = file_header.num_samples_per_trace.value * sample_size_in_bytes
            break
        
        b_trace_data = fobj.read(trc_data_length_in_bytes)

    fobj.close()

INFO:line001_segy_fileheader:number of samples in file header: 1501
INFO:line001_segy_fileheader:sample interval (microseconds) in file header: 2000
INFO:line001_segy_fileheader:Trace length may not be fixed.
INFO:line001_segy_fileheader:number of bytes in each trace: 6004


The file header indicates that the traces may have different trace lengths. However, after scanning all the traces, this is not true. This is just another example of a segy file that is improperly configured.